In [1]:
from PIL import Image
import os
import glob
from keras.preprocessing import image
import keras.preprocessing
import tensorflow as tf

In [3]:
train_gen = image.ImageDataGenerator(
    rescale = 1/255.0,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_gen = image.ImageDataGenerator(rescale=1/255.0)

In [4]:
train_generator = train_gen.flow_from_directory(
    r"D:\\Github\\Oral-cancer\\Phase 1\\Tongue\\Train\\",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 139 images belonging to 2 classes.


In [5]:
train_generator.class_indices

{'Cancer': 0, 'Non Cancer': 1}

In [11]:
val_generator = test_gen.flow_from_directory(
    r"D:\\Github\\Oral-cancer\\Phase 1\\Tongue\\Val\\",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary')

Found 8 images belonging to 2 classes.


In [6]:
# Create the VGG16 models

from keras.applications.vgg16 import VGG16
import keras
from keras.layers import *
from keras.models import Model 
import matplotlib.pyplot as plt

In [7]:
model = VGG16(include_top=False, input_shape=(224,224,3))

In [8]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(2,activation='softmax')(d1)
model_new = Model(model.input,fc2)
model_new.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model_new.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])

In [16]:
hist = model_new.fit_generator(
    train_generator,
    steps_per_epoch = 8,
    epochs = 10,
    validation_data = val_generator,
    validation_steps = 2,
)

C:\Users\anupa\AppData\Local\Temp\ipykernel_19620\1201473955.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model_new.fit_generator(


Epoch 1/10


ValueError: in user code:

    File "C:\Users\anupa\anaconda3\envs\Oral\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\anupa\anaconda3\envs\Oral\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\anupa\anaconda3\envs\Oral\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\anupa\anaconda3\envs\Oral\lib\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "C:\Users\anupa\anaconda3\envs\Oral\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\anupa\anaconda3\envs\Oral\lib\site-packages\keras\losses.py", line 141, in __call__
        losses = call_fn(y_tru